In [1]:

import numpy as np
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import libs.visualization_package as vis
plt.rcParams['figure.figsize'] = (10,8)

In [2]:
def visualize_accuracy_full(history_normal, history_cascade, cascasde_path):
  test_epoch_start = history_cascade["Epoch_test_start"].values
  normall_acc = history_normal["accuracy"].values[0]
  normal_x = np.arange(0, normall_acc.shape[0],1)
  for i in np.unique(test_epoch_start):
    indexes = np.argwhere(test_epoch_start == i)
    cascade_epochs = 50-i-1
    learning_types = history_cascade["Cascade_type"].values[indexes]
    cascade_acc = history_cascade["accuracy"].values[indexes]
    cascade_x = np.arange(0, cascade_epochs,1) + i + 1
    plt.plot(normal_x, normall_acc, label="Normal training")
    for j in range(cascade_acc.shape[0]):
      results_acc = cascade_acc[j][0]
      if cascade_x.shape[0] > results_acc.shape[0]:
        diff_results = cascade_x.shape[0]- results_acc.shape[0]
        for k in range(diff_results):
          results_acc = np.append(results_acc, cascade_acc[j][0][-5+k])
      plt.plot(cascade_x, results_acc, label=learning_types[j][0])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    plt.savefig(os.path.join(cascasde_path[:-24], f'accuracy_{i}.png'))
    plt.clf()


def visualize_loss_full(history_normal, history_cascade, cascade_path):
  test_epoch_start = history_cascade["Epoch_test_start"].values
  normall_acc = history_normal["loss"].values[0]
  normal_x = np.arange(0, normall_acc.shape[0],1)
  for i in np.unique(test_epoch_start):
    indexes = np.argwhere(test_epoch_start == i)
    cascade_epochs = 50-i-1
    learning_types = history_cascade["Cascade_type"].values[indexes]
    cascade_acc = history_cascade["loss"].values[indexes]
    cascade_x = np.arange(0, cascade_epochs,1) + i + 1
    plt.plot(normal_x, normall_acc, label="Normal training")
    for j in range(cascade_acc.shape[0]):
      results_acc = cascade_acc[j][0]
      if cascade_x.shape[0] > results_acc.shape[0]:
        diff_results = cascade_x.shape[0]- results_acc.shape[0]
        for k in range(diff_results):
          results_acc = np.append(results_acc, cascade_acc[j][0][-5+k])
      plt.plot(cascade_x, results_acc, label=learning_types[j][0])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    plt.savefig(os.path.join(cascade_path[:-24], f'loss{i}.png'), transparent=False)
    plt.clf()

In [37]:
def extract_normal_history(df_path):
    normal_history = {}
    normal_data = pd.read_parquet(df_path)
    normal_history["loss"] = normal_data["loss"]
    normal_history['val_loss'] = normal_data["val_loss"]
    normal_history['accuracy'] = normal_data["accuracy"]
    normal_history['val_accuracy'] = normal_data["val_accuracy"]
    return normal_history

def extract_cascade_history(df_path):
    cascade_history = {}
    cascade_data = pd.read_parquet(df_path)
    cascade_history["Epoch_test_start"] = cascade_data["Epoch_test_start"]
    cascade_history["loss"] = cascade_data["loss"]
    cascade_history["val_loss"] = cascade_data["val_loss"]
    cascade_history["accuracy"] = cascade_data["accuracy"]
    cascade_history["val_accuracy"] = cascade_data["val_accuracy"]
    cascade_history["Cascade_type"] = cascade_data["Cascade_type"]
    return cascade_history    

def draw_plots(df_normal, df_cascade):
    normal_history = extract_normal_history(df_normal)
    cascade_histories = extract_cascade_history(df_cascade)
    visualize_accuracy_full(normal_history, cascade_histories, df_cascade)
    visualize_loss_full(normal_history, cascade_histories, df_cascade)

df_normal = "/workplace/tests/2022-08-02 19:19:24.980896_VGG1616_CIFAR100/results/normal_training.parquet"
df_cascade = "/workplace/tests/2022-08-03 07:15:52.900234_VGG1616_CIFAR100/results/cascade_training.parquet"
draw_plots(df_normal, df_cascade)

<Figure size 720x576 with 0 Axes>

In [44]:
df_CIFAR100 = pd.read_parquet("/workplace/tests/2022-07-27 19:48:40.769633_ResNet32_CIFAR100/results/normal_training.parquet")
df_EMNIST = pd.read_parquet("/workplace/tests/2022-07-29 20:40:53.609600_ResNet32_EMNIST/results/normal_training.parquet")
df_MNIST = pd.read_parquet("/workplace/tests/2022-07-30 08:08:07.365100_ResNet32_MNIST/results/normal_training.parquet")
total_df = pd.concat([df_MNIST, df_EMNIST, df_CIFAR100]).reset_index(drop=True)
total_df["Model"] = "ResNet50"
total_df[["Model", "training_time", "training_time_epoch", "batch_size", "Test_acc", "trainable_params", "non_trainable_params", "total_params"]]

,Model,training_time,training_time_epoch,batch_size,Test_acc,trainable_params,non_trainable_params,total_params
0,ResNet50,972.229384,19.444588,32,0.97400,24259082,53120,24312202
1,ResNet50,5935.262707,118.705254,32,0.72871,24272446,53120,24325566
2,ResNet50,9684.825859,193.696517,32,0.30340,24282212,53120,24335332


In [51]:
ResNet_df = pd.read_parquet("/workplace/tests/2022-07-30 08:08:07.365100_ResNet32_MNIST/results/normal_training.parquet")
SimpleDNN_df = pd.read_parquet("/workplace/tests/2022-07-30 07:48:14.462222_SimpleDNN32_MNIST/results/normal_training.parquet")
VGG16_df = pd.read_parquet("/workplace/tests/2022-07-30 09:19:29.157269_VGG1632_MNIST/results/normal_training.parquet")
total_df = pd.concat([SimpleDNN_df, VGG16_df, ResNet_df])
total_df["Model"] = ["SimpleDNN", "VGG16", "ResNet50"]
total_df["Dataset"] = "MNIST"
total_df["batch_size"] = [32,32,32]
total_df[["Model", "Dataset","training_time", "training_time_epoch", "batch_size", "Test_acc", "trainable_params", "non_trainable_params", "total_params"]]


,Model,Dataset,training_time,training_time_epoch,batch_size,Test_acc,trainable_params,non_trainable_params,total_params
0,SimpleDNN,MNIST,119.807606,2.396152,32,0.952,729866,0.0,729866.0
0,VGG16,MNIST,552.422000,11.048440,32,0.962,15045962,0.0,15045962.0
0,ResNet50,MNIST,972.229384,19.444588,32,0.974,24259082,53120.0,24312202.0


In [13]:
base_df = pd.read_parquet("/workplace/tests/2022-08-02 19:19:24.980896_VGG1616_CIFAR100/results/normal_training.parquet")
cascade_df = pd.read_parquet("/workplace/tests/2022-08-03 07:15:52.900234_VGG1616_CIFAR100/results/cascade_training.parquet")
cascade_df["Model"] = "VGG16"
cascade_df["Dataset"] = "CIFAR100"
cascade_df["Reference_acc"] = base_df["Test_acc"]
cascade_df["Reference_training_time"] = base_df["training_time"]
cascade_df["Reference_training_time_epoch"] = base_df["training_time_epoch"]
show_columns = ['Model', 'Dataset', 'Epoch_test_start', 'Cascade_type', 'training_time', 'Reference_training_time','training_time_epoch', 
                'Reference_training_time_epoch', 'Test_acc', 'Reference_acc']
cascade_df["Epoch_test_start"] = cascade_df["Epoch_test_start"].values +1
cascade_df["training_time_epoch"] = cascade_df["training_time"].values / (np.array([50,50,50,50,50,50])-cascade_df["Epoch_test_start"].values)
cascade_df[show_columns]

,Model,Dataset,Epoch_test_start,Cascade_type,training_time,Reference_training_time,training_time_epoch,Reference_training_time_epoch,Test_acc,Reference_acc
0,VGG16,CIFAR100,15,FBCL,1262.521603,1904.254904,36.072046,38.085098,0.01,0.01
0,VGG16,CIFAR100,15,BFCL,1259.387099,1904.254904,35.982489,38.085098,0.01,0.01
0,VGG16,CIFAR100,15,RSCL,1259.218268,1904.254904,35.977665,38.085098,0.01,0.01
0,VGG16,CIFAR100,30,FBCL,720.136545,1904.254904,36.006827,38.085098,0.01,0.01
0,VGG16,CIFAR100,30,BFCL,718.793997,1904.254904,35.939700,38.085098,0.01,0.01
0,VGG16,CIFAR100,30,RSCL,721.580445,1904.254904,36.079022,38.085098,0.01,0.01


In [10]:
model.load_weights(os.path.join(CHECKPOINT_PATH, f"14.ckpt"))
prediction = np.argmax(model.predict(X_test), axis=1)
metrics2 = accuracy_score(y_test, prediction)
metrics2

0.2282

In [11]:
model.load_weights(os.path.join(CHECKPOINT_PATH, f"29.ckpt"))
prediction = np.argmax(model.predict(X_test), axis=1)
metrics2 = accuracy_score(y_test, prediction)
metrics2

0.3072